# Initialize

In [1]:
from IPython.display import display
import pandas as pd
from db_utils import db_connect, execute_operation
from sql_queries import tables

In [2]:
table_names = [t['name'] for t in tables]

In [3]:
conn, cur = db_connect()
conn.autocommit = True

# `users` table

In [4]:
# Since in the songplay fact table we are going to use only events where a song
# was played ("page" = "NextSong"), let's see if there are users who didn't play
# any song.

# Conclusion: yes, there is one user. Since we want all users in the "users"
# table, let's keep this user.

pd.read_sql(con=conn, sql='''
WITH

users_not_played_songs AS (
  SELECT DISTINCT(userId) FROM staging_events WHERE page != 'NextSong' AND userId IS NOT NULL
),

users_played_songs AS (
  SELECT DISTINCT(userId) FROM staging_events WHERE page = 'NextSong' AND userId IS NOT NULL
)

SELECT * FROM users_not_played_songs
MINUS
SELECT * FROM users_played_songs
;''')

,userid
0,21


In [5]:
# Let's see if there are users who changed level (between "free" and "paid").

# Conclusion: yes, there are 8 users. To keep the "users" table with unique
# "user_id", let's choose their latest level.

pd.read_sql(con=conn, sql='''
SELECT userId, COUNT(*) AS count
FROM (SELECT DISTINCT userId, level
      FROM staging_events
      WHERE userId IS NOT NULL)
GROUP BY 1
HAVING count > 1
ORDER BY userId;''')

,userid,count
0,15,2
1,16,2
2,29,2
3,36,2
4,49,2
5,80,2
6,85,2
7,88,2


# `songs` table

In [6]:
# Let's see if there are song IDs that appear more than once in the
# "staging_songs" table.

# Conclusion: no song appears more than once.

pd.read_sql(con=conn, sql='''
SELECT song_id, COUNT(*) AS count
FROM staging_songs
GROUP BY 1
HAVING count > 1
ORDER BY 1;''')

,song_id,count


# `artists` table

In [7]:
# Let's see if there are artist IDs that appear more than once in the
# "staging_songs" table.

# Conclusions:
# - "artist_name": 396 artist IDs have different names, usually associated with songs with other invited artists --> let's choose the minimum of the name, possibly leaving only the original artist
# - "artist_latitude" and "artist_longitude": 19 artists have both missing values and numeric values --> let's choose the numeric values (using MAX will work)
#                                              6 artists have 2 distinct numeric values --> let's choose the maximum value (arbitrary)
# - "artist_location": 18 artists have one text value and one missing value --> let's choose the numeric values (using MAX will work)
#                      16 artists have two distinct text values --> let's choose the maximum value (arbitrary)

columns = ('artist_name', 'artist_latitude', 'artist_longitude', 'artist_location')

for c in columns:
    print(80 * '-')
    df = pd.read_sql(con=conn, sql=f'''
    WITH

    artists_repeated_values AS (
      SELECT artist_id, COUNT(*) AS repeated, COUNT({c:s}) AS non_null
      FROM (SELECT DISTINCT artist_id, {c:s}
            FROM staging_songs
            WHERE artist_id IS NOT NULL)
      GROUP BY 1
      HAVING repeated > 1)

    SELECT DISTINCT
      ss.artist_id,
      artists_repeated_values.repeated,
      artists_repeated_values.non_null,
      LISTAGG(DISTINCT {c:s}, ', ') WITHIN GROUP(ORDER BY {c:s}) OVER (PARTITION BY artist_id) AS {c:s}
    FROM staging_songs AS ss
    INNER JOIN artists_repeated_values USING (artist_id)
    ORDER BY artist_id;''')
    print(f'{c:s}: {df.shape[0]:d} rows')
    display(df)

--------------------------------------------------------------------------------
artist_name: 396 rows


,artist_id,repeated,non_null,artist_name
0,AR03BDP1187FB5B324,2,2,"Britney Spears, Britney Spears feat. Pharrell ..."
1,AR040M31187B98CA41,2,2,"The Bug Featuring Ricky Ranking, The Bug Featu..."
2,AR04S8J1187FB48358,2,2,"Clifford Brown, Clifford Brown / Max Roach Qui..."
3,AR065TW1187FB4C3A5,3,3,"Nearly God, Tricky, Tricky / The Mad Dog Reflex"
4,AR07SOR1187FB46179,2,2,"Goo Goo Dolls, The Goo Goo Dolls"
...,...,...,...,...
391,ARZER7I1187FB385AF,2,2,"Dr. Dre, Ice Cube ft. Dr. Dre"
392,ARZHCKQ1187B9BA5BA,2,2,"Jazzanova, Jazzanova / Paul Randolph"
393,ARZQ7QA1187B9AD08B,2,2,"Prince, Prince & Tevin Campbell"
394,ARZQYSZ1187FB3AC39,2,2,"Ghostface, Ghostface Killah"


--------------------------------------------------------------------------------
artist_latitude: 25 rows


,artist_id,repeated,non_null,artist_latitude
0,AR065TW1187FB4C3A5,2,1,51.43558
1,AR1CD5Z1187B98F2C2,2,2,"38.91391, 40.76596"
2,AR1OGXT1187B9893EB,2,1,36.34253
3,AR5LMPY1187FB573FE,2,1,41.88415
4,AR5PFDG1187FB5027E,2,1,51.57198
5,AR98JLC1187B9ADE23,2,1,34.05349
6,ARBFQAE1187FB3E957,2,2,"33.95246, 40.65507"
7,ARCPAUB11FF10D8539,2,1,50.94224
8,ARFACVS1187B98C226,2,1,38.91391
9,ARFL99B1187B9A2A45,2,1,33.74831


--------------------------------------------------------------------------------
artist_longitude: 25 rows


,artist_id,repeated,non_null,artist_longitude
0,AR065TW1187FB4C3A5,2,1,-2.57518
1,AR1CD5Z1187B98F2C2,2,2,"-95.3756, -94.37517"
2,AR1OGXT1187B9893EB,2,1,-85.29014
3,AR5LMPY1187FB573FE,2,1,-87.63241
4,AR5PFDG1187FB5027E,2,1,0.46694
5,AR98JLC1187B9ADE23,2,1,-118.24532
6,ARBFQAE1187FB3E957,2,2,"-83.98799, -73.94888"
7,ARCPAUB11FF10D8539,2,1,-2.64647
8,ARFACVS1187B98C226,2,1,-94.37517
9,ARFL99B1187B9A2A45,2,1,-84.39111


--------------------------------------------------------------------------------
artist_location: 34 rows


,artist_id,repeated,non_null,artist_location
0,AR1CD5Z1187B98F2C2,2,2,"Lee's Summit, MO, Shenandoah, IA"
1,AR1OGXT1187B9893EB,2,1,"Overton County, TN"
2,AR5LMPY1187FB573FE,2,1,"Chicago, IL"
3,AR5PFDG1187FB5027E,2,1,"Basildon, Essex, England"
4,AR98JLC1187B9ADE23,2,2,"Los Angeles, CA, Los Angeles, California, Unit..."
5,ARATR0U1187B9B76D0,2,2,"San Francisco CA, San Francisco, CA"
6,ARBFQAE1187FB3E957,2,2,"Brooklyn, NY, LAWRENCEVILLE, Georgia"
7,ARCPAUB11FF10D8539,2,1,"Yeovil, England"
8,ARF2SVO1187FB53E8F,2,1,"Decatur, IL"
9,ARFACVS1187B98C226,2,1,"Lee's Summit, MO"


# `songplays` table

"songplays" table must have non-null values of "userId" and "ts" (start time).
And only rows associated with song plays are relevant ("page" = "NextSong")

Let's try to find matching songs in the staging tables:
- "staging_events" has columns:
  - Artist information: "artist" (artist name)
  - Song information: "length" (song duration), "song" (song title)
- "staging_songs" has columns:
  - Artist information: "artist_id", "artist_latitude", "artist_longitude", "artist_location", "artist_name"
  - Song information: "song_id", "title", "duration", "year"

The values that may match are:
- `staging_events.artist` = `staging_songs.artist_name`
- `staging_events.length` = `staging_songs.duration`
- `staging_events.song` = `staging_songs.title`

In [8]:
# It is mandatory to match the song title and artist name. Let's see if the
# durations match.

# Conclusion: 9 songs don't have a matching duration. So let's remove that
# condition from the JOIN clause.

pd.read_sql('''
SELECT DISTINCT
  s.title,
  s.artist_name,
  s.duration,
  e.length,
  ABS(s.duration - e.length) AS diff_song_duration
FROM staging_events AS e
INNER JOIN staging_songs AS s
   ON (s.title = e.song AND s.artist_name = e.artist)
--   ON (s.title = e.song AND s.artist_name = e.artist AND s.duration = e.length)
WHERE e.page = 'NextSong'
  AND e.userId IS NOT NULL
  AND e.ts IS NOT NULL
  AND diff_song_duration > 0
ORDER BY diff_song_duration DESC;''', con=conn)

,title,artist_name,duration,length,diff_song_duration
0,These Words,Natasha Bedingfield,377.25995,216.65914,160.60081
1,Truly_ Madly_ Deeply,Cascada,367.80363,302.91546,64.88817
2,Heads Will Roll,Yeah Yeah Yeahs,280.55465,220.96934,59.58531
3,Song Sung Blue,Neil Diamond,172.38159,194.40281,22.02122
4,Born To Be Wild,Steppenwolf,211.85261,208.14322,3.70939
5,Pienso En Ti,Shakira,148.19220,145.84118,2.35102
6,Warning,Incubus,291.81342,289.54077,2.27265
7,If I Ain't Got You,Alicia Keys,231.18322,229.06730,2.11592
8,Fortunate Fool,Jack Johnson,228.70159,228.83220,0.13061


# Count rows

In [9]:
print('Number of rows')
print('--------------')

for t in table_names:
    cur.execute(f'SELECT COUNT(*) FROM {t:s};')
    print('{table:14s} = {rows:d}'.format(table=t, rows=cur.fetchone()[0]))
print()
cur.execute("SELECT COUNT(*) FROM staging_events WHERE page = 'NextSong';")
print('Number of rows in "staging_events" associated with song plays = {rows:d}'
      ''.format(rows=cur.fetchone()[0]))

Number of rows
--------------
staging_events = 8056
staging_songs  = 14896
users          = 97
songs          = 14896
artists        = 9553
time           = 6813
songplays      = 333

Number of rows in "staging_events" associated with song plays = 6820


In [10]:
comparisions = (
    {'table1': 'staging_events', 'col': 'userId', 'table2': 'users'},
    {'table1': 'staging_songs', 'col': 'song_id', 'table2': 'songs'},
    {'table1': 'staging_songs', 'col': 'artist_id', 'table2': 'artists'},
    {'table1': 'staging_events', 'col': 'ts', 'table2': 'time'},
)

for comparision in comparisions:
    table1 = comparision['table1']
    col = comparision['col']
    table2 = comparision['table2']

    cur.execute(f'SELECT COUNT(DISTINCT {col:s}) FROM {table1:s};')
    print(f'Number of distinct {table2:s} in "{table1:s}" = {cur.fetchone()[0]:d}')

    if table1 == 'staging_events':
        cur.execute(f"SELECT COUNT(DISTINCT {col:s}) FROM {table1:s} WHERE "
                    "page = 'NextSong';")
        print(f'Number of distinct {table2:s} in "{table1:s}" associated with '
              f'song plays = {cur.fetchone()[0]:d}')
    
    print('(the value must be the same as the number of rows in table '
          f'"{table2:s}")')
    print()

Number of distinct users in "staging_events" = 97
Number of distinct users in "staging_events" associated with song plays = 96
(the value must be the same as the number of rows in table "users")

Number of distinct songs in "staging_songs" = 14896
(the value must be the same as the number of rows in table "songs")

Number of distinct artists in "staging_songs" = 9553
(the value must be the same as the number of rows in table "artists")

Number of distinct time in "staging_events" = 8023
Number of distinct time in "staging_events" associated with song plays = 6813
(the value must be the same as the number of rows in table "time")



# First few rows of all tables (staging tables and star schema tables)

In [11]:
for t in table_names:
    print(80 * '-')
    print(t)
    display(pd.read_sql(f'SELECT * FROM {t:s} LIMIT 10;', con=conn))

--------------------------------------------------------------------------------
staging_events


,artist,auth,firstname,gender,iteminsession,lastname,length,level,location,method,page,registration,sessionid,song,status,ts,useragent,userid
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,52,None,307,1541207073796,None,NaN
1,None,Logged In,Celeste,F,1,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,52,None,200,1541207123796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53.0
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Playa Haters,200,1541207150796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53.0
3,Taylor Swift,Logged In,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,You Belong With Me,200,1541207316796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53.0
4,Amy Winehouse,Logged In,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Valerie,200,1541207546796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53.0
5,Jimmy Eat World,Logged In,Celeste,F,5,Williams,285.83138,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Dizzy,200,1541207775796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53.0
6,None,Logged Out,None,None,0,None,NaN,free,None,GET,Home,NaN,18,None,200,1541239749796,None,NaN
7,Maldita Nerea,Logged In,Anabelle,F,0,Simpson,241.16200,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,158,Supelicula,200,1541254670796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69.0
8,Fluke,Logged In,Connar,M,0,Moreno,478.92853,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540824e+12,168,Bermuda,200,1541257880796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10)...",62.0
9,Habib KoitÃÂ©,Logged In,Jayden,M,0,Fox,285.15220,free,"New Orleans-Metairie, LA",PUT,NextSong,1.541034e+12,185,Din Din Wo,200,1541259368796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101.0


--------------------------------------------------------------------------------
staging_songs


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARXR32B1187FB57099,NaN,NaN,None,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
1,1,ARC43071187B990240,NaN,NaN,"Wisner, LA",Wayne Watson,SOKEJEJ12A8C13E0D0,The Urgency (LP Version),245.21098,0
2,1,AR5AA4Q1187FB4CFBD,NaN,NaN,None,Alisha's Attic,SOPOQFU12A6D4F8C5F,Sex Is On Everyone's Tongue,198.55628,2001
3,1,ARFVYJI1187B9B8E13,47.60356,-122.32944,"Seattle, WA",Pearl Jam,SOAOJYY12A58A7B2F9,Not For You,352.88771,1994
4,1,ARNV5OQ1187FB531D5,NaN,NaN,None,Bandabardò,SOBBHII12AB0184B2C,La mauvaise réputation,138.57914,2008
5,1,AR1WWVL1187B9B0306,NaN,NaN,"Birmingham, England",UB40,SOZODBG12A6701C5D1,Reasons (Radio Remix),207.30730,0
6,1,ARUWLCL1187FB549B8,NaN,NaN,None,Yamandu Costa,SOFGHAM12AB0186412,Mafuá,190.04036,0
7,1,AR1JRJ61187B9B3F37,NaN,NaN,"Berlin, Germany",Hardfloor,SOOQFOY12AB018A713,Don?t Trust Chief Wiggum,437.60281,0
8,1,AREFNKX1187B991576,40.71455,-74.00712,NY - New York City,Dan Zanes,SOZPZUI12A8C133200,The Colorado Trail,72.90730,0
9,1,ARGQJWL1187FB3CE9E,NaN,NaN,None,La Polla Records,SOXLGTI12A8C13C08B,Cara Al Culo,82.99057,1985


--------------------------------------------------------------------------------
users


...

,artist_id,name,location,latitude,longitude
0,ARYO9BU1187B9ADA88,Porcupine Tree,None,NaN,NaN
1,ARRBHUK1187FB4D1EF,Joey Beltram,"Queens, NY",40.70380,-73.83168
2,ARSK10P1187B9B717C,Bobby McFerrin,PULLMAN,46.73016,-117.18148
3,ARBVASN1187B9890CB,Lucky Boys Confusion,"Chicago, IL",NaN,NaN
4,ARYAUMZ1187B9A2A40,Spandau Ballet,"London, UK",51.50632,-0.12714
5,ARCLYBR1187FB53913,Neal Schon,"San Mateo, CA",37.54703,-122.31483
6,AROJZNF1187B9897FD,Ne Luumaet,None,NaN,NaN
7,AR52SSB1187B9B3E5F,Aquagen,None,NaN,NaN
8,ARSL5SP1187B9A7AE0,N.W.A.,"Compton, California, USA.",34.05349,-118.24532
9,ARGEKDX1187FB3A5BF,Marcus Miller,"Brooklyn, NY",40.65507,-73.94888


--------------------------------------------------------------------------------
time


,start_time,hour,day,week,month,year,weekday
0,2018-11-03 01:16:15,1,3,44,11,2018,6
1,2018-11-03 15:52:36,15,3,44,11,2018,6
2,2018-11-03 16:07:39,16,3,44,11,2018,6
3,2018-11-03 16:24:53,16,3,44,11,2018,6
4,2018-11-03 16:39:49,16,3,44,11,2018,6
5,2018-11-03 16:53:35,16,3,44,11,2018,6
6,2018-11-03 17:02:24,17,3,44,11,2018,6
7,2018-11-03 17:09:52,17,3,44,11,2018,6
8,2018-11-03 17:17:26,17,3,44,11,2018,6
9,2018-11-03 17:25:40,17,3,44,11,2018,6


--------------------------------------------------------------------------------
songplays


,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,14,2018-11-20 17:46:38,49,paid,SOCHRXB12A8AE48069,ARTDQRC1187FB4EFD4,758,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...
1,30,2018-11-21 18:29:12,97,paid,SOSMXVH12A58A7CA6C,AR6PJ8R1187FB5AD70,817,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
2,46,2018-11-26 13:31:57,36,paid,SOVPSWY12A58A7B83F,ARF91NB1187B98BDB8,808,"Janesville-Beloit, WI","""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537...."
3,62,2018-11-29 16:09:24,49,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,1041,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...
4,78,2018-11-25 16:14:24,49,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,923,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...
5,94,2018-11-21 12:10:49,15,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,764,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
6,110,2018-11-22 01:59:04,15,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
7,126,2018-11-13 16:55:21,29,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,486,"Atlanta-Sandy Springs-Roswell, GA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
8,142,2018-11-15 19:06:33,97,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,605,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
9,158,2018-11-15 10:52:20,80,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,611,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


# Close connection

In [12]:
conn.close()